# Jaeger

## Принцип работы

- OpenTracing

#### Как формируется TraceID

TraceID формирует первый сервис, который получает запрос. Запрос -- есть, TraceID -- нет => Я первый, создаю TraceID, пробрасываю его дальше по стеку вызовов

#### Семплинг

- Constant (sampler.type=const) sampler always makes the same decision for all traces. It either samples all traces (sampler.param=1) or none of them (sampler.param=0).
- Probabilistic (sampler.type=probabilistic) sampler makes a random sampling decision with the probability of sampling equal to the value of sampler.param property. For example, with sampler.param=0.1 approximately 1 in 10 traces will be sampled.
- Rate Limiting (sampler.type=ratelimiting) sampler uses a leaky bucket rate limiter to ensure that traces are sampled with a certain constant rate. For example, when sampler.param=2.0 it will sample requests with the rate of 2 traces per second.

В elasticsearch трейсы хранятся за все время и не удаляются. На каждый день создается отдельный индекс, например jaeger-service-2019-03-04

### Компоненты:

![](https://www.jaegertracing.io/img/architecture-v2.png)

### Работы библиотеки

![](https://www.jaegertracing.io/img/context-prop.png)

#### Пример сырого спана, который отправляется в collector
```json
{
    "traceID": "442f29763a5d5949",
    "spanID": "1091d145293efd83",
    "flags": 1,
    "operationName": "FindDriverIDs",
    "references": [
        {
            "refType": "CHILD_OF",
            "traceID": "442f29763a5d5949",
            "spanID": "6276a66bc9bdcd28"
        }
    ],
    "startTime": 1561974767517423,
    "duration": 25984,
    "tags": [
        {
            "key": "param.location",
            "type": "string",
            "value": "577,322"
        },
        {
            "key": "span.kind",
            "type": "string",
            "value": "client"
        },
        {
            "key": "internal.span.format",
            "type": "string",
            "value": "proto"
        }
    ],
    "logs": [
        {
            "timestamp": 1561974767543269,
            "fields": [
                {
                    "key": "event",
                    "type": "string",
                    "value": "Found drivers"
                },
                {
                    "key": "level",
                    "type": "string",
                    "value": "info"
                }
            ]
        }
    ],
    "processID": "p1",
    "warnings": null
}
                
```

### Формат логов

Jaeger components only log to standard out, using structured logging library go.uber.org/zap configured to write log lines as JSON encoded strings, for example:

```json
{"level":"info","ts":1517621222.261759,"caller":"healthcheck/handler.go:99","msg":"Health Check server started","http-port":14269,"status":"unavailable"}
```



## Проблемы

- Виснет frontend
- Очень много мусора в результатах
- Collector -- узкое место
- Выполнять фильтрацию можно только по тегам
- Асинхронные Python библиотеки не поддерживают OpenTracing => Trace будет обрываться на момент обращения к ресурсу

## Как интегрировать в проект
- Интегрировать проекты от головы
- Вместо логов -- собирать спаны || логи в формате zap
- Продолжение TraceID из промежуточных систем. Пример: Собирать Span Context из RabbitMQ
- Соблюдать OpenTracing спецификацию

## nginx

**jaeger-config.json**
```json
{
  "service_name": "nginx",
  "diabled": false,
  "reporter": {
    "logSpans": true,
    "localAgentHostPort": "jaeger:6831"
  },
  "sampler": {
    "type": "const",
    "param": "1"
  }
}

```
**nginx.conf**
```nginx
load_module modules/ngx_http_opentracing_module.so;

events {}

http {
  opentracing on;

  opentracing_load_tracer /usr/local/lib/libjaegertracing_plugin.so /etc/jaeger-config.json;
  upstream backend {
    server app-service:9001;
  }

  server {
    error_log /var/log/nginx/debug.log debug;
    listen 8080;
    server_name localhost;

    location = / {
      opentracing_trace_locations off;
      proxy_pass http://backend;
      opentracing_propagate_context;
    }
  }
}

```

## Django

```bash
pip install django_opentracing jaeger-client
```

**setting**

```python
OPENTRACING_TRACE_ALL = True
OPENTRACING_TRACED_ATTRIBUTES = ['META']
OPENTRACING_TRACER_CALLABLE = __name__ + '.tracer'

def tracer():
    from jaeger_client import Config
    config = Config(
        config={ # usually read from some yaml config
            'sampler': {
                'type': 'const',
                'param': 1,
            },
            'logging': True,
        },
        service_name='example')
    return config.initialize_tracer()

MIDDLEWARE = [
    'django_opentracing.OpenTracingMiddleware',
    ...
]
```

## Tornado

```bash
pip install tornado_opentracing jaeger-client
```

```python
from tornado.ioloop import IOLoop
from tornado.web import Application, RequestHandler

import opentracing
import tornado_opentracing


tornado_opentracing.init_tracing()

from jaeger_client import Config
config = Config(
    config={ # usually read from some yaml config
        'sampler': {
            'type': 'const',
            'param': 1,
        },
        'logging': True,
    },
    service_name='tornado')


tracer = config.initialize_tracer()

class MainHandler(RequestHandler):
    def get(self):
        self.write({'status': 'ok'})


class StoryHandler(RequestHandler):
    def get(self, story_id):
        if int(story_id) == 0:
            raise ValueError('invalid value passed')
        self.write({'status': 'fetched'})


app = Application([
        (r'/', MainHandler),
        (r'/story/([0-9]+)', StoryHandler),
    ],
    opentracing_tracer=tornado_opentracing.TornadoTracer(tracer),
    opentracing_trace_all=True,
    opentracing_traced_attributes=['protocol', 'method'],
)
app.listen(8080)
IOLoop.current().start()
```


## aiohttp

```bash
pip install aiozipkin
```

```python
async def make_app(host, port):
    app = web.Application()
    app.router.add_get('/', handle)
    # here we aquire reference to route, so later we can command
    # aiozipkin not to trace it
    skip_route = app.router.add_get('/status', not_traced_handle)

    endpoint = az.create_endpoint('aiohttp_server', ipv4=host, port=port)

    zipkin_address = 'http://127.0.0.1:9411/api/v2/spans'
    tracer = await az.create(zipkin_address, endpoint, sample_rate=1.0)
    az.setup(app, tracer, skip_routes=[skip_route])
    return app


def run():
    host = '127.0.0.1'
    port = 9001
    loop = asyncio.get_event_loop()
    app = loop.run_until_complete(make_app(host, port))
    web.run_app(app, host=host, port=port)
```

Пример записи спана:
```python
tracer = az.get_tracer(request.app)
span = az.request_span(request)

with tracer.new_child(span.context) as child_span:
    child_span.name('mysql:select')
    # call to external service like https://python.org
    # or database query
    await asyncio.sleep(0.01)
```

## Ссылки

Библиотеки:

https://github.com/opentracing-contrib/nginx-opentracing

https://github.com/opentracing-contrib/python-django

https://github.com/opentracing-contrib/python-tornado

https://github.com/aio-libs/aiozipkin ( Совместимая с Jaeger opentracing клиент для asyncio )

https://github.com/jaegertracing/jaeger-client-python ( Библиотека для создания Jaeger трейсера )

https://github.com/jaegertracing/jaeger-operator/ ( Kubernetes оператор )

Статьи:

https://eng.uber.com/distributed-tracing/

Видео:

https://www.youtube.com/watch?v=fsHb0qK37bc ( Общее введение, общая концепция, Русский)

https://www.youtube.com/watch?v=fjYAU3jayVo ( Видео Тимлида Jeager, Введение + Demo / Программисту )

https://www.youtube.com/watch?v=qg0ENOdP1Lo ( Доклад с конференции -- Jeager в kubernetes / Админу )

Материалы по глубже:

https://storage.googleapis.com/pub-tools-public-publication-data/pdf/36356.pdf

https://www.shkuro.com/books/2019-mastering-distributed-tracing/

Документация/спецификация:

https://github.com/opentracing/specification/blob/master/specification.md ( Описание понятий Opentracing )

https://github.com/opentracing/specification/blob/master/semantic_conventions.md ( Соглашение по семантике и модели данных спана )

https://opentracing.io/docs/overview/ 

https://medium.com/opentracing/take-opentracing-for-a-hotrod-ride-f6e3141f7941

https://medium.com/opentracing/how-to-enable-nginx-for-distributed-tracing-9479df18b22c ( Небольшой пример настройки nginx )

Туториалы: 

https://www.katacoda.com/courses/opentracing/golang-hotrod-demo ( Самостоятельный запуск Jeager )

https://github.com/yurishkuro/opentracing-tutorial/tree/master/python ( Примеры работы с opentracing в Python )
    